<a href="https://colab.research.google.com/github/aymenchibouti/doctorat/blob/main/lstm_smote_rendemzesearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Load the files
enrollment_df = pd.read_csv('/mnt/data/enrollment_train.csv')
log_df = pd.read_csv('/mnt/data/log_train spliting.csv')
truth_df = pd.read_csv('/mnt/data/truth_train.csv', header=None, names=['enrollment_id', 'dropout'])

# Preprocess log file
log_df['time'] = pd.to_datetime(log_df['time'])
min_time_per_enrollment = log_df.groupby('enrollment_id')['time'].min().reset_index().rename(columns={'time': 'start_time'})
log_df = log_df.merge(min_time_per_enrollment, on='enrollment_id')
log_df['day'] = (log_df['time'] - log_df['start_time']).dt.days + 1
log_df.drop(columns=['start_time'], inplace=True)

# Create event count per day
log_df['count'] = 1
pivot_df = log_df.pivot_table(index=['enrollment_id', 'day'], columns='event', values='count', aggfunc='sum', fill_value=0).reset_index()
pivot_wide_df = pivot_df.pivot_table(index='enrollment_id', columns='day')
pivot_wide_df.columns = [f"{event}_day{day}" for (event, day) in pivot_wide_df.columns]
pivot_wide_df = pivot_wide_df.reset_index()

# Merge all data
merged_df = enrollment_df.merge(truth_df, on='enrollment_id', how='left')
final_df = merged_df.merge(pivot_wide_df, on='enrollment_id', how='left')

# Prepare X and y
X = final_df.drop(columns=['enrollment_id', 'username', 'course_id', 'dropout']).fillna(0)
y = final_df['dropout']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle imbalanced data using SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_res, y_res = smote.fit_resample(X_scaled, y)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Define function to create the LSTM model
def create_lstm_model(units=64, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=(X_train.shape[1], 1)))  # Reshape for LSTM (timesteps, features)
    model.add(LSTM(units, activation='tanh', return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units // 2, activation='tanh'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Convert model to KerasClassifier
model = KerasClassifier(build_fn=create_lstm_model, verbose=0)

# Define parameter grid for RandomizedSearchCV
param_dist = {
    'units': randint(64, 256),  # Number of units in the LSTM layers
    'dropout_rate': [0.2, 0.3, 0.4],  # Dropout rate
    'learning_rate': [0.001, 0.0005, 0.01],  # Learning rate
    'batch_size': [16, 32, 64],  # Batch size
    'epochs': [10, 20, 30]  # Number of epochs
}

# Define RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, verbose=2, random_state=42)

# Fit the model
random_search.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])), y_train)

# Get the best parameters
print(f"Best parameters found: {random_search.best_params_}")

# Evaluate the best model
best_model = random_search.best_estimator_
y_pred_prob = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1])))
y_pred = (y_pred_prob > 0.5).astype(int)

# Evaluate model performance
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Plot ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

# Plot Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()
